In [ ]:
#  Copyright 2017-2021 Reveal Energy Services, Inc
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This file is part of Orchid and related technologies.
#

# Example: Using Pandas to Analyze Completion Parameters

This notebook illustrates using the Orchid* Python API and the pandas package to
perform completion analysis.

(*Orchid is a mark of Reveal Energy Services, Inc)

## 0.5 Import packages

The only import needed for the Python API is `orchid` itself.

In [ ]:
import orchid

The remaining imports are standard python packages to support the analysis.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# The following import is included for its "side-effects" of an improved color schemes and
# plot styles. (See the "Tip" in section 9.2 of "Python for Data Analysis" for details.)
import seaborn as sns
from scipy import integrate

## 1.0 Load the .ifrac project

The following code simply captures the configured location of the Orchid training data. It is not needed to
use the Orchid Python API itself, but it is used in this example to load well-known data.

In [ ]:
orchid_training_data_path = orchid.training_data_path()

In [ ]:
bakken_project = orchid.load_project(str(orchid_training_data_path.joinpath(
    'frankNstein_Bakken_UTM13_FEET.ifrac')))

## 2.0 Define a function to compute the stage treatment data

In [ ]:
def compute_stage_treatment_aggregates(treatment_stage):
    # These calculations IGNORE all calculation warnings.
    pumped_fluid_volume = orchid.pumped_fluid_volume(treatment_stage,
                                                     treatment_stage.start_time,
                                                     treatment_stage.stop_time)[0].magnitude
    total_proppant_mass = orchid.total_proppant_mass(treatment_stage,
                                                     treatment_stage.start_time,
                                                     treatment_stage.stop_time)[0].magnitude
    median_treating_pressure = orchid.median_treating_pressure(treatment_stage,
                                                               treatment_stage.start_time,
                                                               treatment_stage.stop_time)[0].magnitude
    return pumped_fluid_volume, total_proppant_mass, median_treating_pressure

## 3.0 Build a pandas data frame

In [ ]:
bakken_units = bakken_project.project_units

In [ ]:
# Calculate the stage results
stage_results = []
for well in bakken_project.wells:
    stages = list(well.stages)

    for stage in stages:
        
        treatment_aggregates = compute_stage_treatment_aggregates(stage)
        # Skip stages with no aggregates. These stages most likely are from 
        # an untreated monitor well.
        if not treatment_aggregates:
            continue
            
        stage_fluid, stage_proppant, median_stage_pressure = treatment_aggregates
        stage_results.append((bakken_project.name, well.name, stage.display_stage_number,
                              stage.md_top(bakken_units.LENGTH).magnitude,
                              stage.md_bottom(bakken_units.LENGTH).magnitude,
                              stage_fluid, stage_proppant, median_stage_pressure))

In [ ]:
# Provide a way to manage DataFrame column names (which include units) using simpler,
# semantic identifiers. Creating the column names helps me avoid "typos" involved in
# getting the (project-specific) units correct, but is not necessary in a typical
# interactive session.

bakken_columns = {'project': 'Project',
                  'well': 'Well',
                  'stage': 'Stage',
                  'md_top': f'MD Top ({orchid.abbreviation(bakken_units.LENGTH)})',
                  'md_bottom': f'MD Bottom ({orchid.abbreviation(bakken_units.LENGTH)})',
                  'total_fluid':
                      f'Total Fluid ({orchid.abbreviation(bakken_project.slurry_rate_volume_unit())})',
                  'total_proppant':
                      f'Total Proppant ({orchid.abbreviation(bakken_project.proppant_concentration_mass_unit())})',
                  'median_treating':
                      f'Median Treating Pressure ({orchid.abbreviation(bakken_units.PRESSURE)})'}

In [ ]:
# Create the data frame
bakken_summaries = pd.DataFrame(data=stage_results,
                                columns=bakken_columns.values())
bakken_summaries.head()

### 3.1 Compute the stage length directly from the data frame

In [ ]:
bakken_summaries[f'Stage Length ({orchid.abbreviation(bakken_units.LENGTH)})'] = \
    bakken_summaries.apply(lambda s: s[bakken_columns['md_bottom']] - s[bakken_columns['md_top']], axis=1)
bakken_columns['stage_length'] = bakken_summaries.columns[-1]
bakken_summaries.head()

### 3.2 Now compute the proppant loading for each stage

In [ ]:
bakken_summaries[f'Proppant loading' \
                 f' ({orchid.abbreviation(bakken_project.proppant_concentration_mass_unit())}' \
                 f'/{orchid.abbreviation(bakken_units.LENGTH)})'] = \
    bakken_summaries.apply(lambda s: s[bakken_columns['total_proppant']] / s[bakken_columns['stage_length']],
                           axis=1)
bakken_columns['proppant_loading'] = bakken_summaries.columns[-1]
bakken_summaries.head()

## 4.0 Completion questions

### 4.1 What is the median proppant intensity per well?

In [ ]:
bakken_summaries[[bakken_columns['well'],
                  bakken_columns['proppant_loading']]]. \
    groupby(bakken_columns['well']).median()

In [ ]:
groups = bakken_summaries.groupby(bakken_columns['well'])

fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group[bakken_columns['stage']],
            group[bakken_columns['proppant_loading']],
            marker='o', linestyle='', ms=6, label=name)
ax.legend()
plt.rcParams['figure.dpi'] = 150
plt.show()

That's a little hard to interpret with the outliers

In [ ]:
# Plot
groups = bakken_summaries[bakken_summaries[bakken_columns['stage']] > 5]. \
    groupby(bakken_columns['well'])

fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group[bakken_columns['stage']],
            group[bakken_columns['proppant_loading']],
            marker='o', linestyle='', ms=6, label=name)
ax.legend()
ax.set_title(f'Proppant Loading by Stage')
ax.set_xlabel('Stage Number')
ax.set_ylabel(bakken_columns['proppant_loading'])
plt.rcParams['figure.dpi'] = 150
plt.show()

That's a little busy. Let's clean it up.

In [ ]:
def build_proppant_loading_plot(data_frame):
    groups = data_frame.groupby(bakken_columns['well'])

    fig, ax = plt.subplots(len(groups), sharex=True, sharey=True)
    fig.suptitle(f'{bakken_columns["proppant_loading"]} by Stage')
    i=0
    colors=['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple',
            'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
    for name, group in groups:
        ax[i].margins(0.05)
        ax[i].plot(group[bakken_columns['stage']],
                   group[bakken_columns['proppant_loading']],
                   marker='o', linestyle='', ms=6, label=name)
        ax[i].legend()
        ax[i].set_xlabel('Stage Number')
        i = i+1
        # Hide x labels and tick labels for all but bottom plot.
    for a in ax:
        a.label_outer()
    plt.rcParams['figure.dpi'] = 150
    plt.show()

build_proppant_loading_plot(bakken_summaries[bakken_summaries[bakken_columns['stage']]>5])

### 4.2 What is the median treating pressure per well?

In [ ]:
bakken_summaries[[bakken_columns['well'],
                  bakken_columns['median_treating']]]. \
    groupby(bakken_columns['well']).median()

In [ ]:
groups = bakken_summaries.groupby(bakken_columns['well'])

fig, ax = plt.subplots(3, sharex=True, sharey=True)
fig.suptitle(f'{bakken_columns["median_treating"]} by Stage')
i=0
colors=['tab:blue', 'tab:orange', 'tab:green',
        'tab:purple', 'tab:yellow', 'tab:red', 'tab:black']
for name, group in groups:
    ax[i].margins(0.05)
    ax[i].plot(group[bakken_columns['stage']],
               group[bakken_columns['median_treating']],
               colors[i], marker='o', linestyle='', ms=6, label=name)
    ax[i].legend()
    ax[i].set_xlabel('Stage Number')
    i = i+1

    # Hide x labels and tick labels for all but bottom plot.
for a in ax:
    a.label_outer()
plt.show()

## 5.0 Add another .ifrac project

In [ ]:
montney_project = orchid.load_project(str(orchid_training_data_path.joinpath(
    'Project-frankNstein_Montney_UTM13_METERS.ifrac')))

In [ ]:
montney_units = montney_project.project_units

In [ ]:
# Calculate the stage results for the Montney project
stage_results = []
for well in montney_project.wells:
    stages = list(well.stages)

    for stage in stages:

        treatment_aggregates = compute_stage_treatment_aggregates(stage)
        # Skip stages with no aggregates. These stages most likely are from
        # an untreated monitor well.
        if not treatment_aggregates:
            continue

        stage_fluid, stage_proppant, median_stage_pressure = treatment_aggregates
        stage_results.append((montney_project.name, well.name, stage.display_stage_number,
                              stage.md_top(montney_units.LENGTH).magnitude,
                              stage.md_bottom(montney_units.LENGTH).magnitude,
                              stage_fluid, stage_proppant, median_stage_pressure))

In [ ]:
# Provide a way to manage DataFrame column names (which include units) using simpler,
# semantic identifiers.

montney_columns = {'project': 'Project',
                  'well': 'Well',
                  'stage': 'Stage',
                  'md_top': f'MD Top ({orchid.abbreviation(montney_units.LENGTH)})',
                  'md_bottom': f'MD Bottom ({orchid.abbreviation(montney_units.LENGTH)})',
                  'total_fluid':
                       f'Total Fluid ({orchid.abbreviation(montney_project.slurry_rate_volume_unit())})',
                  'total_proppant':
                       f'Total Proppant ({orchid.abbreviation(montney_project.proppant_concentration_mass_unit())})',
                  'median_treating':
                       f'Median Treating Pressure ({orchid.abbreviation(montney_units.PRESSURE)})'}

In [ ]:
# Create the Montney data frame
montney_summaries = pd.DataFrame(data=stage_results,
                                columns=montney_columns.values())

montney_summaries[f'Stage Length ({orchid.abbreviation(montney_units.LENGTH)})'] = \
    montney_summaries.apply(
        lambda s: s[montney_columns['md_bottom']] -
                  s[montney_columns['md_top']], axis=1)
montney_columns['stage_length'] = montney_summaries.columns[-1]

montney_summaries[f'Proppant loading' \
                  f' ({orchid.abbreviation(montney_project.proppant_concentration_mass_unit())}/'
                  f'{orchid.abbreviation(montney_units.LENGTH)})'] = \
    montney_summaries.apply(lambda s: s[montney_columns['total_proppant']] / s[montney_columns['stage_length']],
                            axis=1)
montney_columns['proppant_loading'] = montney_summaries.columns[-1]

montney_summaries.head()

### 5.1 Combine the data for the two projects.

Because the frankNstein_Bakken_UTM13_FEET project is in US Oilfield units, and the
Project-frankNstein_Montney_UTM13_METERS project is in metric units, I'll need to
convert one to the other.

In [ ]:
# Calculate the stage results in US oilfield units for the Montney project
stage_results = []
for well in montney_project.wells:
    stages = list(well.stages)

    for stage in stages:

        treatment_aggregates = compute_stage_treatment_aggregates(stage)
        if not treatment_aggregates:
            continue

        # Convert stage aggregates to US oilfield units
        stage_fluid = orchid.to_unit(
            bakken_project.slurry_rate_volume_unit(),
            orchid.Measurement(treatment_aggregates[0],
                               montney_project.slurry_rate_volume_unit())).magnitude
        stage_proppant = orchid.to_unit(
            bakken_project.proppant_concentration_mass_unit(),
            orchid.Measurement(treatment_aggregates[1],
                               montney_project.proppant_concentration_mass_unit())).magnitude
        median_stage_pressure = orchid.to_unit(
            bakken_units.PRESSURE,
            orchid.Measurement(treatment_aggregates[2], montney_units.PRESSURE)).magnitude

        stage_results.append((montney_project.name, well.name,
                              stage.display_stage_number,
                              stage.md_top(bakken_units.LENGTH).magnitude,
                              stage.md_bottom(bakken_units.LENGTH).magnitude,
                              stage_fluid, stage_proppant, median_stage_pressure))

In [ ]:
# Create the converted Montney data frame
montney_us_summaries = pd.DataFrame(data=stage_results,
                                    columns=list(bakken_columns.values())[:-2])

montney_us_summaries[f'Stage Length ({orchid.abbreviation(bakken_units.LENGTH)})'] = \
    montney_us_summaries.apply(
        lambda s: s[bakken_columns['md_bottom']] - s[bakken_columns['md_top']], axis=1)

montney_us_summaries[f'Proppant loading' \
                     f' ({orchid.abbreviation(bakken_project.proppant_concentration_mass_unit())}/'
                     f'{orchid.abbreviation(bakken_units.LENGTH)})'] = \
    montney_us_summaries.apply(lambda s: s[bakken_columns['total_proppant']] / s[bakken_columns['stage_length']],
                               axis=1)

montney_us_summaries.head()

In [ ]:
combined = bakken_summaries[bakken_summaries[bakken_columns['stage']] > 5]\
    .append(montney_us_summaries)
combined.head()

In [ ]:
combined[[bakken_columns['well'], bakken_columns['proppant_loading']]]\
    .groupby(bakken_columns['well'])\
    .median()

In [ ]:
combined[[bakken_columns['well'], bakken_columns['proppant_loading']]]\
    .groupby(bakken_columns['well'])\
    .agg({bakken_columns['proppant_loading']:['median', 'std']})

In [ ]:
build_proppant_loading_plot(combined)